<a href="https://colab.research.google.com/github/SainiManisha/Data-Science_Tutorial/blob/master/Deployment_of_Deep_Learning_and_Machine_Learning_Model_FLASK_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('./drive')

In [ ]:
!pip install flask-ngrok

In [1]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request

import joblib
import pandas as pd

import base64

In [2]:
app = Flask(__name__)
# It will starts ngrok when the app is run
run_with_ngrok(app)

In [3]:
@app.route('/',methods=['GET','POST'])
def predict():
    x = """
    <h1> Model Deployment </h1>
    <ul>
    <li> <a href='/ml'>ML prediction</a> </li>
    <li> <a href='/dl'>DL Prediction</a> </li>
    </ul>
    """
    return x

# ML Model

In [4]:
@app.before_first_request
def load_ml_model_to_app():
    app.ml_model = joblib.load('./drive/My Drive/linear_regression_boston_model.pkl')

@app.route('/ml')
def ml():
    x = """
    <!doctype html>
    <title>ML Form</title>
    <h1>Upload new File</h1>
    <form method=post action='/ml/submit'>
        <input name=CRIM placeholder='per capita crime rate by town'><br>
        <input name=ZN placeholder='proportion of residential land zoned for lots over 25,000 sq.ft'><br>
        <input name=INDUS placeholder='proportion of non-retail business acres per town'><br>
        <input name=CHAS placeholder='Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)'><br>
        <input name=NOX placeholder='nitric oxides concentration (parts per 10 million)'><br>
        <input name=RM placeholder='average number of rooms per dwelling'><br>
        <input name=AGE placeholder='proportion of owner-occupied units built prior to 1940'><br>
        <input name=DIS placeholder='weighted distances to five Boston employment centres'><br>
        <input name=RAD placeholder='index of accessibility to radial highways'><br>
        <input name=TAX placeholder='full-value property-tax rate per $10,000'><br>
        <input name=PTRATIO placeholder='pupil-teacher ratio by town'><br>
        <input name=B placeholder='1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town'><br>
        <input name=LSTAT placeholder='% lower status of the population'><br>
        <input type=submit value=Predict>
    </form>
    """
    return x

@app.route('/ml/submit', methods=['POST'])
def ml_submit():
    column_names = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']
    data = {}
    for key in column_names:
        data[key] = [float(request.form[key])]
    print(data)

    df = pd.DataFrame.from_dict(data)
    price = app.ml_model.predict(df)
    print(price)

    return """<!doctype html>
    <title>BOSTON ML PREDICTION</title>
    Predicted price of home is: """ + "<b>" + str(price[0]) + "</b>" + \
        """<br><a href='/'>Go Back</a>"""

# DL Model

In [5]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

In [6]:
@app.before_first_request
def load_dl_model_to_app():
    app.dl_model = ResNet50(weights='imagenet')

@app.route('/dl')
def dl():
    x = """
    <!doctype html>
    <title>DL Form</title>
    <h1>Upload new File</h1>
    <form method=post action='/dl/submit' enctype='multipart/form-data'>
        <input name=image type=file placeholder='image'><br>
        <input type=submit value=Predict>
    </form>
    """
    return x

@app.route('/dl/submit',methods=['POST'])
def dl_submit():
	img = request.files['image']
	img.save('./myimage')
 
	image = tf.image.decode_image(
		tf.io.read_file('./myimage'))
	image = tf.image.resize(image, [224, 224])
	pre_image = preprocess_input(image)
	# make as 4D
	pre_image = pre_image[None, ...]

	b = tf.image.decode_image(tf.io.read_file("./myimage"))
	jpeg = tf.image.encode_jpeg(b)
	b = base64.b64encode(jpeg.numpy()).decode('utf-8')

	probas = app.dl_model.predict(pre_image)
	x = decode_predictions(probas, top=5)
	print(x)
 
	x = [(i[1], i[2] * 100)for i in x[0]]

	return f"""<!doctype html>
	<title>ResNet50 Image Prediction</title>
	Predicted class image is (with probability): <b>{x}</b> <br>
	<img src='data:image/jpeg;base64,{b}'>
	""" + """<br><a href='/'>Go Back</a>"""

# Run Web Server

In [7]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9fedcaee3200.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [23/Oct/2020 21:46:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2020 21:46:54] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Oct/2020 21:46:55] "GET /ml HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2020 21:46:58] "GET /dl HTTP/1.1" 200 -


127.0.0.1 - - [23/Oct/2020 21:47:10] "POST /dl/submit HTTP/1.1" 200 -


[[('n02093428', 'American_Staffordshire_terrier', 0.5986699), ('n02093256', 'Staffordshire_bullterrier', 0.3057205), ('n02087394', 'Rhodesian_ridgeback', 0.01259152), ('n02100583', 'vizsla', 0.011867594), ('n02108089', 'boxer', 0.007724751)]]


127.0.0.1 - - [23/Oct/2020 21:49:22] "POST /dl/submit HTTP/1.1" 200 -


[[('n04522168', 'vase', 0.22199424), ('n04476259', 'tray', 0.054047767), ('n03938244', 'pillow', 0.05387108), ('n03485794', 'handkerchief', 0.030729977), ('n02776631', 'bakery', 0.029545631)]]


127.0.0.1 - - [23/Oct/2020 21:49:54] "GET / HTTP/1.1" 200 -
